<a href="https://colab.research.google.com/github/recervictory/LearingPython/blob/Student/08%20-%20Pandas%20III%20-%20Data%20Cleaning%20and%20Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning and Preparation

During the course of doing data analysis and modeling, a *significant amount of time* is spent on data preparation: loading, cleaning, transforming, and rearranging. Such tasks are often reported to take up *80% or more of an analyst’s time*.



In [ ]:
import pandas as pd
import numpy as np
from numpy import nan as NA # represent NaN as NA

## A. Handling Missing Data
Missing data occurs commonly in many data analysis applications. One of the goals
of pandas is to make working with missing data as painless as possible. For example,
all of the descriptive statistics on pandas objects exclude missing data by default.

The way that missing data is represented in pandas objects is somewhat imperfect,
but it is functional for a lot of users. For numeric data, pandas uses the floating-point
value NaN (Not a Number) to represent missing data.

The built-in Python **None** value is also treated as NA in object arrays:

In [ ]:
string_data = pd.Series(['Kolkata', 'Delhi', np.nan, 'Bangalore'])
string_data

In [ ]:
string_data.isnull()

In [ ]:
# The built-in Python None value is also treated as NA in object arrays:
string_data[0] = None
string_data.isnull()

### NA handling methods
- `dropna` Filter axis labels based on whether values for each label have missing data, with varying thresholds for how much missing data to tolerate.
- `fillna` Fill in missing data with some value or using an interpolation method such as 'ffill' or 'bfill'.
- `isnull` Return boolean values indicating which values are missing/NA.
- `notnull` Negation of isnull

### Filtering Out Missing Data
While you always have the option to do it by hand using `pandas.isnull` and boolean indexing, the `dropna` can be helpful.

In [ ]:
data = pd.Series([1, NA, 3.5, NA, 7])
data

In [ ]:
# Droping the Data
data.dropna()

In [ ]:
# This is equivalent to:
data[data.notnull()]

With DataFrame objects, things are a bit more complex. You may want to drop rows
or columns that are all NA or only those containing any `NAs`. 
The `dropna` by default drops **any row containing a missing value**:

In [ ]:
 
 data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 6.5, 3.]])
 cleaned = data.dropna()
 cleaned

In [ ]:
# Passing how='all' will only drop rows that are all NA:
data.dropna(how='all')

In [ ]:
# To drop columns in the same way, pass axis=1:
data[4] = NA
data

In [ ]:
# Drop data column wise
data.dropna(axis=1, how='all')

### Filling In Missing Data
For most purposes, the fillna method is the workhorse function to use. Calling fillna with a **constant** replaces **missing values** with that value:

In [ ]:
df = pd.DataFrame(np.random.randn(7, 3), columns=['gold', 'silver', 'copper'])
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

In [ ]:
# Fill The missing values with Zero
df.fillna(0)

In [ ]:
# Calling fillna with a dict, you can use a different fill value for each column:
df.fillna({'silver': -1, 'copper': 1})

In [ ]:
# fillna returns a new object, but you can modify the existing object in-place:
df.fillna(0)
print(df)
df.fillna(0, inplace=True) # Important
print(df)

The same **interpolation** methods available for reindexing can be used with fillna:

In [ ]:
# Creating Dataframe
df = pd.DataFrame(np.random.randn(8, 3), columns=['gold', 'silver', 'copper'])
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

In [ ]:
# Fill 'NA' with forword fill method
df.fillna(method='ffill')

In [ ]:
# limit by row
df.fillna(method='ffill', limit=2)

In [ ]:
# you might pass the mean or median values
df.fillna(df.mean())

In [ ]:
df.fillna(df.mean(),inplace=True)

In [ ]:
df['category'] = [NA,'A','B',NA,'A','C','A','B']
df

In [ ]:
df['category'].fillna(df['category'].mode()[0], inplace=True)
df

## B. Data Transformation

### Removing Duplicates
Duplicate rows may be found in a DataFrame for any number of reasons. Here is an example:

In [ ]:
data = pd.DataFrame({'city': ['kolkata', 'delhi'] * 3 + ['delhi'],'count': [1, 1, 2, 3, 3, 4, 4]})
data

The DataFrame method `duplicated()` returns a **boolean Series** indicating whether each row is a duplicate (has been observed in a previous row) or not:

In [ ]:
data.duplicated()

The `drop_duplicates()` returns a DataFrame where the duplicated array is False:

In [ ]:
data.drop_duplicates()

In [ ]:
data['price'] = np.random.randint(10,100,size=7)
data

In [ ]:
 # Drop duplicate by column
 data.drop_duplicates(['city'])

### Transforming Data Using a Function or Mapping
For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame. 

In [ ]:
data = pd.DataFrame({'city':['New York','Delhi','Kolkata','Chicago','Las Vegas'], 
                     'pupulation': np.random.randint(100000,1000000000,size=5)
                     })
data

In [ ]:
city_to_country = {'new york':'usa','delhi':'india','kolkata':'india','chicago':'usa','las vegas':'usa'}
city_to_country

In [ ]:
# We Need to cheack the data type
data.dtypes

In [ ]:
data['city'] = data['city'].str.lower()
data

In [ ]:
data['country'] = data['city'].map(city_to_country)
data

### Replacing Values
Filling in missing data with the `fillna()` method is a special case of more general value replacement. As you’ve already seen, `map()` can be used to modify a subset of values in an object but replace provides a simpler and more flexible way to do so. 

In [ ]:
data['pupulation'] = data['pupulation'].replace([843448647,127963973	],np.nan)
data

### Detecting and Filtering Outliers
Filtering or transforming **outliers** is largely a matter of applying array operations. Consider a DataFrame with some normally distributed data:

In [ ]:
data = pd.DataFrame(np.random.randn(1000, 4),columns=['Aaba','Baba','Caca','Dada'])

# Lets find out the outliers
data.describe()

In [ ]:
data[np.abs(data['Caca']) > 3]

In [ ]:
# Detecting outleirs from any columns in the dataframe

data[(np.abs(data) > 3).any(1)] # axis = 1 i.e column wise

In [ ]:
data[(np.abs(data) > 3).all(1)]

In [ ]:
new_row = {'Aaba' : 4,	'Baba':4,	'Caca': -4,	'Dada': -4}
data = data.append(new_row,ignore_index=True)
data[(np.abs(data) > 3).all(1)]

# Project: Risk of being drawn into online sex work

### Context
This database was used in the paper: Covert online ethnography and machine learning for detecting individuals at risk of being drawn into online sex work. 2018 IEEE/ACM International Conference on Advances in Social Networks Analysis and Mining (ASONAM), Barcelona, Spain, 28-31 August.

### Content
The database includes data scraped from a European online adult forum. Using covert online ethnography we interviewed a small number of participants and determined their risk to either supply or demand sex services through that forum. This is a great dataset for semi-supervised learning.

### Inspiration
How can we identify individuals at risk of being drawn into online sex work? The spread of online social media enables a greater number of people to be involved into online sex trade; however, detecting deviant behaviors online is limited by the low available of data. To overcome this challenge, we combine covert online ethnography with semi-supervised learning using data from a popular European adult forum.

## Importing Data

In [ ]:
import pandas as pd
import numpy as np

import warnings; warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/content/online_sex_work.csv', index_col=0)
df = df.iloc[: 28831, :]

df.head()

In [ ]:
# Understand the Data Types
df.dtypes

## Data Cleaning


### Change datatype for some features

Data in a number of features that contain numerical data could be converted into pure numbers (integers), which would take less memory and could be interpreted more easily by machine learning models.

In [ ]:
df.index = df.index.astype(int)
df['Number_of_advertisments_posted'] = df['Number_of_advertisments_posted'].astype(int)
df['Number_of_offline_meetings_attended'] = df['Number_of_offline_meetings_attended'].astype(int)
df['Profile_pictures'] = df['Profile_pictures'].astype(int)
df['Friends_ID_list'] = df['Friends_ID_list'].astype(str)
df['Risk'] = df['Risk'].astype(str)

df.head()

In [ ]:
df.dtypes

In [ ]:
# cheack the Error
# df['Number_of_Comments_in_public_forum'] = df['Number_of_Comments_in_public_forum'].astype(int)

In [ ]:
df['Number_of_Comments_in_public_forum'] = df['Number_of_Comments_in_public_forum'].str.replace(' ', '').astype(int)

### Counting the Missing Values

In [ ]:
# Count of missing values column wise
df.isnull().sum()

### Convert `Gender` to binary data

In the `Gender` column, We fill some missing values using some simple conditions (if the entry is, for example, homosexual, and looking for men, we fill that entry with `male`), using the `fill_gender_na` function below. Then in every entry, we change the data to whether it specifies `female` or not.

In [ ]:
def fill_gender_na(row):
    if row['Sexual_orientation'] == 'Homosexual':
        if row['Looking_for'] == 'Men':
            return 'male'
        elif row['Looking_for'] == 'Women':
            return 'female'
    elif row['Sexual_orientation'] == 'Heterosexual':
        if row['Looking_for'] == 'Men':
            return 'female'
        elif row['Looking_for'] == 'Women':
            return 'male'
    return np.nan

In [ ]:
## Fill the missing data
fill_values = df.apply(fill_gender_na, axis=1)
df['Gender'].fillna(fill_values, inplace=True)

In [ ]:
# Lets check the missing values
df.isnull().sum()

In [ ]:
# Add missing value with summary statistics 
df['Gender'].fillna(df['Gender'].mode()[0], inplace=True)
df.head()

In [ ]:
# Lets check the missing values
df.isnull().sum()

### Insert new Binary column named 'Female'

In [ ]:
df.insert(0, 'Female', df['Gender'] == 'female')
df.head()

### Missing values in `Location`

In [ ]:
df['Location'].fillna(df['Location'].mode()[0], inplace=True)

### Decimal points in `Age`

We replace all commas (European decimal separator) with periods, while handling some unformatted values.

In [ ]:
def comma_replace(obj):
  return obj.replace(",",".")

df['Age'].head().apply(comma_replace)

In [ ]:
# Lets do with single line with lambda
df['Age'] = df['Age'].apply(lambda obj: obj.replace(',', '.'))
df.head()

In [ ]:
# Error: Convering age to numeric
pd.to_numeric(df['Age'])

In [ ]:
# Method 1
df['Age'] = df['Age'].replace('???', np.nan)
df['Age'] = df['Age'].astype(float)

In [ ]:
# Method 2
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')

In [ ]:
# Lets check the missing values
df.isnull().sum()

In [ ]:
df['Age'].fillna(df['Age'].mean(), inplace=True)

In [ ]:
df.isnull().sum()

### Convert `Verification` to binary data

In every entry, we change the data to whether the user is verified or not.

In [ ]:
df['Verification'] = df['Verification'] != 'Non_Verified'
df[['Verification']].head()